<a href="https://colab.research.google.com/github/Ucchwas/ECG-Classification-using-CNN/blob/main/RNN_LSTM_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

In [81]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/mitbih_train_Ihua.csv')

In [82]:
df.drop('1',axis='columns',inplace=True)
#print(df.dtypes)
df.fillna(0, inplace=True)
X = df.drop('Label1',axis='columns')
#Xs = df.drop('Label1',axis='columns')
#X = Xs[df['Label1'] != 1 & 2 & 3]
y = testLabels = df.Label1.astype(np.float32)
#ys = testLabels = df.Label1.astype(np.float32)
#y = ys[df['Label1'] != 1 & 2 & 3]

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,train_size=0.8, random_state=15, stratify=y)
#print(y_train.value_counts())
#print(y.value_counts())
#print(y.value_counts())
#print(len(X_train.columns))

#from tensorflow_addons import losses
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [84]:
from keras.layers import Input, Convolution1D, BatchNormalization, MaxPooling1D, Flatten, Dense
from keras.models import Model

In [92]:
def RNN(X_train, y_train, X_test, y_test, loss, weights):
    input_shape = (X_train.shape[1], 1)

    # Create the sequential model
    model = tf.keras.models.Sequential()

    # Add the LSTM layer
    lstm_layer = LSTM(64, activation='relu', input_shape=input_shape)
    model.add(lstm_layer)

    # Add the output layer
    output_layer = Dense(5, activation='softmax')
    model.add(output_layer)

    # Compile the model
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    # Train the model
    if weights == -1:
        if len(X_train) == 0:
            print("The training data is empty.")
        else:
            model.fit(X_train, y_train, epochs=50)
    else:
        if len(X_train) == 0:
            print("The training data is empty.")
        else:
            model.fit(X_train, y_train, epochs=50, class_weight=weights)

    # Evaluate the model
    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_train)
    y_preds = np.argmax(y_preds, axis=1)

    print("Classification Report: \n", classification_report(y_train, y_preds))
    return model, y_preds


In [86]:
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [87]:
from keras.utils import to_categorical

# Convert y_train to one-hot encoding
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [93]:
model,history=RNN(X_train,y_train,X_test,y_test, 'binary_crossentropy', -1)

Epoch 1/50
2189/2189 [==============================] - 80s 36ms/step - loss: 0.0517 - accuracy: 0.9608
Epoch 2/50
2189/2189 [==============================] - 79s 36ms/step - loss: 0.0281 - accuracy: 0.9796
Epoch 3/50
2189/2189 [==============================] - 77s 35ms/step - loss: 0.0217 - accuracy: 0.9837
Epoch 4/50
2189/2189 [==============================] - 76s 35ms/step - loss: 0.0179 - accuracy: 0.9865
Epoch 5/50
2189/2189 [==============================] - 77s 35ms/step - loss: 0.0157 - accuracy: 0.9879
Epoch 6/50
2189/2189 [==============================] - 77s 35ms/step - loss: 0.0136 - accuracy: 0.9892
Epoch 7/50
2189/2189 [==============================] - 80s 36ms/step - loss: 0.0114 - accuracy: 0.9906
Epoch 8/50
2189/2189 [==============================] - 76s 35ms/step - loss: 0.0100 - accuracy: 0.9916
Epoch 9/50
2189/2189 [==============================] - 76s 35ms/step - loss: 0.0093 - accuracy: 0.9923
Epoch 10/50
2189/2189 [==============================] - 75s 34m

In [94]:
model.save("RNN_LSTM_Multiclass.h5")

In [95]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("RNN_LSTM_Multiclass.h5")

In [96]:
# convert to tensorflow lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("RNN_LSTM_Multiclass.tflite", "wb").write(tflite_model)

482756